In [1]:
# !pip install mecab-python3
#These wheels include a copy of the MeCab library, but not a dictionary. 
#In order to use MeCab you'll need to install a dictionary. unidic-lite is a good one to start with:
# !pip install unidic-lite

In [2]:
# normalization tool
# !pip install neologdn

In [3]:
# !pip install openpyxl

In [4]:
# # To be able to see Japanese!
# !pip install japanize_matplotlib

everything was deleted what a christmas miracle....
lets start again ...


## Librarys

In [3]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

import os
import pandas as pd

# Preprocessing
import MeCab
import neologdn
import collections
from nltk import FreqDist
from nltk.corpus import stopwords

# Visualization
import matplotlib.pyplot as plt
import japanize_matplotlib
#import seaborn as sns # REMINDER: make sure to remove if not using!

In [4]:
file_name = 'Combined_85K_150.xlsx'
df = pd.read_excel(r'/home/samuelhenderson/code/simply-japanese/data/2_RawData/' + file_name)
df

,#日本語(原文),#やさしい日本語,#英語(原文)
0,彼女は通りを横切った。,彼女は通りを横に通っていった。,she came across the street .
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。,"as far as i know , she is a very good person ."
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。,all the girls in my class are kind .
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。,he couldn 't pass the examination .
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。,"she 'll be glad to see you , won 't she ?"
...,...,...,...
145,いとこは私を町中案内してくれた。,いとこは私に町の中を案内してくれた。,My cousin took me around the town.
146,木の葉が赤や黄色になり始めた。,森が赤い色や黄色になりはじめた。,The leaves began to turn red and yellow.
147,今頃ロンドンは寒いに違いない。,今の季節ロンドンは寒いに違いない。,London must be cold now.
148,このスープは何か足した方がいいんじゃないかしら。,このスープは何か加えたほうがいいのではないですか。,This soup needs something to pick it up.


In [5]:
df.drop(columns=['#英語(原文)'], inplace=True)
df.head()

,#日本語(原文),#やさしい日本語
0,彼女は通りを横切った。,彼女は通りを横に通っていった。
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。


In [8]:
df.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
df.head()

,original,simplified
0,彼女は通りを横切った。,彼女は通りを横に通っていった。
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。


In [9]:
# cosine similarity 
# X= df['original']
# y= df['simplified']

# cosine_similarity(X, y)

In [10]:
#testing out stuff from github repository : 
# https://github.com/ZHAOTING/Japanese-sentence-similarity-scorer

In [11]:
!python -m unidic download


/home/samuelhenderson/.pyenv/versions/simply-japanese/bin/python: No module named unidic


In [12]:
# #fast text
# import fasttext
# import fasttext.util

In [13]:
fasttext.util.download_model('jp', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

NameError: name 'fasttext' is not defined

## SumEval & Bleu caluculator (Sacrebleu)
https://github.com/chakki-works/sumeval#welcome-contribution-tada

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

Trying to get the right dependency for the sumeval & Bleu caluculator (Sacrebleu)

In [ ]:
 # pip install sumeval==0.1.5

In [ ]:
# pip uninstall sumeval

In [ ]:
# pip install janome

In [ ]:
import sacrebleu

In [ ]:
from sacrebleu import corpus_bleu, DEFAULT_TOKENIZER

In [ ]:
type(df.iloc[0][0])

In [ ]:
from sumeval.metrics.bleu import BLEUCalculator
# https://github.com/chakki-works/sumeval
from janome.tokenizer import Tokenizer

# bleu calculator for english
# bleu = BLEUCalculator()
# score = bleu.bleu("I am waiting on the beach",
#                   "He is walking on the beach")

# bleu calculator for Japanese
bleu_ja = BLEUCalculator(lang="ja")
score_ja = bleu_ja.bleu(df.iloc[0][0], df.iloc[0][1])
tokenizer_ja()

In [18]:
from sumeval.metrics.bleu import BLEUCalculator

bleu = BLEUCalculator()
score = bleu.bleu("I am waiting on the beach",
                  "He is walking on the beach")

# bleu_ja = BLEUCalculator(lang="ja")
# score_ja = bleu_ja.bleu("私はビーチで待ってる", "彼がベンチで待ってる")

In [15]:
from sacrebleu import ja_mecab

ImportError: cannot import name 'ja_mecab' from 'sacrebleu' (/home/samuelhenderson/.pyenv/versions/3.8.12/envs/simply-japanese/lib/python3.8/site-packages/sacrebleu/__init__.py)

In [ ]:
bleu = BLEUCalculator()

In [1]:
df.iloc[0][0]

NameError: name 'df' is not defined

In [17]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu_ja = BLEUCalculator(lang="ja")
score_ja = bleu_ja.bleu(df.iloc[0][0], df.iloc[0][1])


TypeError: tokenizer_ja() missing 1 required positional argument: 'text'

In [ ]:
import sacrebleu

In [ ]:
sacrebleu.info()
# https://github.com/chakki-works/sumeval#welcome-contribution-tada

In [16]:
from sacrebleu.metrics import BLEU, CHRF, TER

refs = [ # First set of references
          ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.'],
          # Second set of references
          ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.'],
        ]
sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

bleu = BLEU()

bleu.corpus_score(sys, refs)
# BLEU = 48.53 82.4/50.0/45.5/37.5 (BP = 0.943 ratio = 0.944 hyp_len = 17 ref_len = 18)

bleu.get_signature()
# nrefs:2|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0

chrf = CHRF()

chrf.corpus_score(sys, refs)
# chrF2 = 59.73

TypeError: __init__() missing 1 required positional argument: 'args'

In [ ]:
# !pip list

## WER (Word Error Rate) and Edit Distance
WER - https://en.m.wikipedia.org/wiki/Word_error_rate 
WER= {S+D+I}/{N}={S+D+I}/{S+D+C}
where:

S is the number of substitutions,
D is the number of deletions,
I is the number of insertions,
C is the number of correct words,
N is the number of words in the reference (N=S+D+C)

Edit Distance (Levenshtein distance) https://en.m.wikipedia.org/wiki/Edit_distance
Takes two either been an insert/remove/replace of characters.

i.e. 
Input:   str1 = “geek”, str2 = “gesek”

Output:  1

Explanation: We can convert str1 into str2 by inserting a ‘s’.

In [44]:
# A WER function I found online.
# Seems be going off legth of characters in simplification process,
# But in a fair few of the conversions the phrases get longer.

def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#correct words " + str(numCor))
        print("#substitutions " + str(numSub))
        print("#deleltions " + str(numDel))
        print("#insertions " + str(numIns))
    # return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'numCor':numCor, 'numSub':numSub, 'numIns':numIns, 'numDel':numDel, "numCount": len(r)}

In [45]:
# Testing first sample
wer(df.iloc[0][0], df.iloc[0][1])

OP	REF	HYP
SUB	彼女は通りを横切った。	彼女は通りを横に通っていった。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0


{'WER': 1.0, 'numCor': 0, 'numSub': 1, 'numIns': 0, 'numDel': 0, 'numCount': 1}

In [169]:
# Testing Sample 2
wer(df.iloc[2][0], df.iloc[2][1])

OP	REF	HYP
OK	私のクラスの少女たちはみんな親切だ。	私のクラスの少女たちはみんな親切だ。
#correct words 1
#substitutions 0
#deleltions 0
#insertions 0


{'WER': 0.0, 'numCor': 1, 'numSub': 0, 'numIns': 0, 'numDel': 0, 'numCount': 1}

In [47]:
len(df.iloc[2][0]), len(df.iloc[2][1])

(18, 18)

In [48]:
for i in range(150):
    if len(df.iloc[i][0])> len(df.iloc[i][1]):
        print(i, len(df.iloc[i][0]), len(df.iloc[i][1]))

1 17 16
9 18 17
12 10 9
14 23 21
27 18 17
34 12 10
48 22 19
56 17 15
60 12 11
71 19 18
72 17 16
80 20 17
84 11 10
86 19 16
87 21 17
90 21 16
94 13 12
95 17 15
96 9 8
102 13 12
105 23 20
106 19 14
109 35 33
112 18 17
118 20 18
121 27 24
125 20 19
130 15 12
135 19 16
136 17 12
138 20 19
144 27 24
149 36 17


In [55]:
print('Testing WER on simplified sentences with smaller outputs than input...')
for i in range(150):    
    if len(df.iloc[i][0])> len(df.iloc[i][1]):
        print('Sample', i+1)
        print('Difference in Characters = ', len(df.iloc[i][0]) - len(df.iloc[i][1]))
        print(len(df.iloc[i][0]) - len(df.iloc[i][1]))
        wer(df.iloc[i][0], df.iloc[i][1])

Testing WER on simplified sentences with smaller outputs than input...
Sample 2
Difference in Characters =  1
1
OP	REF	HYP
SUB	私が知るかぎり彼女は大変よい人だ。	私が知る限り彼女は大変よい人だ。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0
Sample 10
Difference in Characters =  1
1
OP	REF	HYP
SUB	そのドレスはあなたによく似合います。	そのドレスはあなたによく合います。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0
Sample 13
Difference in Characters =  1
1
OP	REF	HYP
SUB	暮らしはどうですか。	生活はどうですか。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0
Sample 15
Difference in Characters =  2
2
OP	REF	HYP
SUB	彼女には私とほとんど同じぐらいチャンスがある。	彼女には私とほとんど同じぐらい機会がある。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0
Sample 28
Difference in Characters =  1
1
OP	REF	HYP
SUB	赤ちゃんは泣きながら眠ってしまった。	赤ちゃんは泣きながら寝てしまった。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0
Sample 35
Difference in Characters =  2
2
OP	REF	HYP
SUB	どこで髪の毛を切ったの。	どこで髪を切ったの。
#correct words 0
#substitutions 1
#deleltions 0
#insertions 0
Sample 49
Differ

In [57]:
i = 10
wer('I love to dance and sing', 'I love singing and i love to do dancing')

OP	REF	HYP
OK	I	I
OK	love	love
INS	****	singing
INS	****	and
INS	****	i
SUB	to	love
SUB	dance	to
SUB	and	do
SUB	sing	dancing
#correct words 2
#substitutions 4
#deleltions 0
#insertions 3


{'WER': 1.167,
 'numCor': 2,
 'numSub': 4,
 'numIns': 3,
 'numDel': 0,
 'numCount': 6}

In [58]:
df.iloc[i][0]

'次に何をしたらよいのですか。'

In [107]:
wer(ori, simpi)

OP	REF	HYP
OK	彼	彼
OK	女	女
OK	は	は
OK	通	通
OK	り	り
OK	を	を
OK	横	横
INS	****	に
INS	****	通
INS	****	っ
INS	****	て
SUB	切	い
OK	っ	っ
OK	た	た
OK	。	。
#correct words 10
#substitutions 1
#deleltions 0
#insertions 4


{'WER': 0.455,
 'numCor': 10,
 'numSub': 1,
 'numIns': 4,
 'numDel': 0,
 'numCount': 11}

In [ ]:
wer()

In [167]:
n = 0
original_text = list(df.iloc[n][0]) 
simplified_text = list(df.iloc[n][1])

def wer_jp(original, simplified):       
    ori = ''
    simpi = ''

    for i in original_text:
        ori += i + ' '
    
    for i in simplified_text:
        simpi += i + ' '
    
    return wer(ori, simpi)

for i in np.arange(0, 149):
    original_text = list(df.iloc[i][0]) 
    simplified_text = list(df.iloc[i][1])
    print(' ')
    print('******************')
    print('Testing sample ', i+1)
    print('******************')
    wer_jp(original_text, simplified_text)

 
******************
Testing sample  1
******************
OP	REF	HYP
OK	彼	彼
OK	女	女
OK	は	は
OK	通	通
OK	り	り
OK	を	を
OK	横	横
INS	****	に
INS	****	通
INS	****	っ
INS	****	て
SUB	切	い
OK	っ	っ
OK	た	た
OK	。	。
#correct words 10
#substitutions 1
#deleltions 0
#insertions 4
 
******************
Testing sample  2
******************
OP	REF	HYP
OK	私	私
OK	が	が
OK	知	知
OK	る	る
DEL	か	****
SUB	ぎ	限
OK	り	り
OK	彼	彼
OK	女	女
OK	は	は
OK	大	大
OK	変	変
OK	よ	よ
OK	い	い
OK	人	人
OK	だ	だ
OK	。	。
#correct words 15
#substitutions 1
#deleltions 1
#insertions 0
 
******************
Testing sample  3
******************
OP	REF	HYP
OK	私	私
OK	の	の
OK	ク	ク
OK	ラ	ラ
OK	ス	ス
OK	の	の
OK	少	少
OK	女	女
OK	た	た
OK	ち	ち
OK	は	は
OK	み	み
OK	ん	ん
OK	な	な
OK	親	親
OK	切	切
OK	だ	だ
OK	。	。
#correct words 18
#substitutions 0
#deleltions 0
#insertions 0
 
******************
Testing sample  4
******************
OP	REF	HYP
OK	彼	彼
OK	は	は
OK	試	試
OK	験	験
OK	に	に
OK	合	合
OK	格	格
OK	で	で
OK	き	き
OK	な	な
OK	か	か
OK	っ	っ
OK	た	た
OK	。	。
#correct words 14
#substitutions 0
#deleltions 0
#insertions 0
 


In [158]:
i = 4
wer_jp((df.iloc[i][0]), len(df.iloc[i][1]))

OP	REF	HYP
OK	彼	彼
OK	女	女
OK	は	は
OK	通	通
OK	り	り
OK	を	を
OK	横	横
INS	****	に
INS	****	通
INS	****	っ
INS	****	て
SUB	切	い
OK	っ	っ
OK	た	た
OK	。	。
#correct words 10
#substitutions 1
#deleltions 0
#insertions 4


{'WER': 0.455,
 'numCor': 10,
 'numSub': 1,
 'numIns': 4,
 'numDel': 0,
 'numCount': 11}